# L'algorithme quantique HHL avec Qiskit
Ce notebook illustre une implémentation de l'algorithme HHL (Harrow, Hassidim, Lloyd) pour résoudre un système linéaire.

In [6]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from scipy.linalg import expm
from qiskit.quantum_info import Statevector

# Système à résoudre : A x = b
A = np.array([[1, 1], [1, 3]], dtype=float)
b = np.array([1, 0], dtype=float)
b = b / np.linalg.norm(b)

# Décalage spectral pour rendre A définie positive avec valeurs propres > 1
eigvals, eigvecs = np.linalg.eigh(A)
min_eig = np.min(eigvals)
alpha = 1.1 - min_eig if min_eig < 1.1 else 0  # On veut min eig > 1
A_shifted = A + alpha * np.eye(2)
eigvals, eigvecs = np.linalg.eigh(A_shifted)

n_phase = 1
n_ancilla = 1
n_solution = 1
n_qubits = n_phase + n_ancilla + n_solution

qc = QuantumCircuit(n_qubits)

theta = 2 * np.arccos(b[0])
qc.ry(theta, 2)
qc.h(0)

t = np.pi / (eigvals[1] - eigvals[0])
U = expm(1j * A_shifted * t)
U_gate = UnitaryGate(U)
qc.append(U_gate.control(1), [0,2])

qc.h(0)

for idx, eig in enumerate(eigvals):
    val = 1 / eig
    if np.abs(val) > 1:
        val = np.sign(val)
    angle = -2 * np.arcsin(val)
    if idx == 0:
        qc.cry(angle, 0, 1)
    else:
        qc.x(0)
        qc.cry(angle, 0, 1)
        qc.x(0)

state = Statevector.from_instruction(qc).data

print("Amplitudes non nulles :")
for idx, amp in enumerate(state):
    if np.abs(amp) > 1e-6:
        print(format(idx, f'0{n_qubits}b'), amp)

# Extraction de la solution (phase et ancilla à 0)
solution = []
for idx, amp in enumerate(state):
    binaire = format(idx, f'0{n_qubits}b')
    if binaire[:2] == '00':
        solution.append(amp)
solution = np.array(solution)
norm = np.linalg.norm(solution)
if norm > 0:
    solution = solution / norm
else:
    print("Aucune amplitude extraite, vérifiez la correspondance phase/valeur propre et la QPE.")

print("Vecteur solution normalisé (approximation HHL Qiskit, matrice décalée) :")
print(solution)
print("Solution classique (matrice décalée) :", np.linalg.solve(A_shifted, np.array([1, 0])))

Amplitudes non nulles :
000 (0.6004481466768739+0.32129454620568054j)
001 (0.157931525571582-0.13841005364568815j)
010 (-0.15805350407468977-0.08457304623045642j)
011 (-0.34463484311982323+0.3020354989401261j)
100 (-0.11691897192583796-0.3212945462056801j)
101 (0.050367369653683304+0.138410053645688j)
110 (0.03077610166333569+0.0845730462304563j)
111 (-0.10991061142563122-0.3020354989401258j)
Vecteur solution normalisé (approximation HHL Qiskit, matrice décalée) :
[0.84255864+0.45084575j 0.2216121 -0.19421925j]
Solution classique (matrice décalée) : [ 0.81323632 -0.23141346]


# Régression linéaire quantique 2x2 avec HHL

Ce bloc de code montre comment utiliser l’algorithme HHL pour approximer la solution d’un problème de régression linéaire sur deux points, c’est-à-dire résoudre (A x = b) où (A) est la matrice des covariances et (b) le vecteur des ordonnées.  
On prépare l’état quantique correspondant à (b), puis on applique les étapes principales de HHL : estimation de phase, rotation conditionnelle, et extraction de la solution.  
La solution quantique est comparée à la solution classique obtenue par inversion de la matrice (A).


In [11]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from scipy.linalg import expm
from qiskit.quantum_info import Statevector

# Cas concret : régression linéaire quantique sur deux points
A = np.array([[5, 2], [2, 2]], dtype=float)
b = np.array([8, 3], dtype=float)
b = b / np.linalg.norm(b)

eigvals, eigvecs = np.linalg.eigh(A)

n_phase = 1
n_ancilla = 1
n_solution = 1
n_qubits = n_phase + n_ancilla + n_solution

qc = QuantumCircuit(n_qubits)

# Préparation de |b> sur le qubit solution (qubit 2)
theta = 2 * np.arccos(b[0])
qc.ry(theta, 2)

# QPE (Hadamard sur le qubit de phase)
qc.h(0)

# Controlled-U = exp(i*A*t)
t = np.pi / (eigvals[1] - eigvals[0])
U = expm(1j * A * t)
U_gate = UnitaryGate(U)
qc.append(U_gate.control(1), [0,2])

# Inverse QPE (Hadamard)
qc.h(0)

# Rotation conditionnelle sur l'ancilla selon la valeur propre
for idx, eig in enumerate(eigvals):
    val = 1 / eig
    if np.abs(val) > 1:
        val = np.sign(val)
    angle = -2 * np.arcsin(val)
    if idx == 0:
        qc.cry(angle, 0, 1)
    else:
        qc.x(0)
        qc.cry(angle, 0, 1)
        qc.x(0)

state = Statevector.from_instruction(qc).data

print("Amplitudes non nulles :")
for idx, amp in enumerate(state):
    if np.abs(amp) > 1e-6:
        print(format(idx, f'0{n_qubits}b'), amp)

# Extraction de la solution (phase et ancilla à 0)
solution = []
for idx, amp in enumerate(state):
    binaire = format(idx, f'0{n_qubits}b')
    if binaire[:2] == '00':
        solution.append(amp)
solution = np.array(solution)
norm = np.linalg.norm(solution)
if norm > 0:
    solution = solution / norm
else:
    print("Aucune amplitude extraite, vérifiez la correspondance phase/valeur propre et la QPE.")

print("Vecteur solution normalisé (approximation HHL Qiskit, régression linéaire) :")
print(solution)

# Solution classique
sol_classique = np.linalg.solve(A, np.array([8, 3]))
print("Solution classique :", sol_classique)

# Normalisation de la solution classique pour comparaison
sol_classique_norm = sol_classique / np.linalg.norm(sol_classique)
print("Solution classique normalisée :", sol_classique_norm)

Amplitudes non nulles :
000 (0.12550646862220435-0.24419823901393j)
010 (-0.02121446519040141+0.04127703613996683j)
011 (-0.8090423864266355-0.24766221683980105j)
100 (-0.041630779580614716-0.15601554159223338j)
110 (0.007036886098044315+0.026371439756089972j)
111 (-0.3933447581766571-0.1582286385365398j)
Vecteur solution normalisé (approximation HHL Qiskit, régression linéaire) :
[4.57114122e-01-8.89408050e-01j 1.80430794e-16+5.52330645e-17j]
Solution classique : [ 1.66666667 -0.16666667]
Solution classique normalisée : [ 0.99503719 -0.09950372]


# Simulation de réseau électrique avec HHL

On cherche les potentiels aux nœuds d’un petit réseau de résistances, modélisé par la matrice de Laplace ( L ).

## Problème :
Soit un réseau à 2 nœuds reliés par des résistances, avec une source de courant injectée au nœud 1 et retirée au nœud 2.

[ L V = I ]

In [12]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from scipy.linalg import expm
from qiskit.quantum_info import Statevector

# Matrice de Laplace du réseau (2 nœuds)
A = np.array([[2, -1], [-1, 1]], dtype=float)
b = np.array([1, -1], dtype=float)
b = b / np.linalg.norm(b)  # Préparation quantique

# Décalage spectral pour garantir des valeurs propres > 1
eigvals, eigvecs = np.linalg.eigh(A)
min_eig = np.min(eigvals)
alpha = 1.1 - min_eig if min_eig < 1.1 else 0
A_shifted = A + alpha * np.eye(2)
eigvals, eigvecs = np.linalg.eigh(A_shifted)

n_phase = 1
n_ancilla = 1
n_solution = 1
n_qubits = n_phase + n_ancilla + n_solution

qc = QuantumCircuit(n_qubits)
theta = 2 * np.arccos(b[0])
qc.ry(theta, 2)
qc.h(0)

t = np.pi / (eigvals[1] - eigvals[0])
U = expm(1j * A_shifted * t)
U_gate = UnitaryGate(U)
qc.append(U_gate.control(1), [0,2])
qc.h(0)

for idx, eig in enumerate(eigvals):
    val = 1 / eig
    if np.abs(val) > 1:
        val = np.sign(val)
    angle = -2 * np.arcsin(val)
    if idx == 0:
        qc.cry(angle, 0, 1)
    else:
        qc.x(0)
        qc.cry(angle, 0, 1)
        qc.x(0)

state = Statevector.from_instruction(qc).data

print("Amplitudes non nulles :")
for idx, amp in enumerate(state):
    if np.abs(amp) > 1e-6:
        print(format(idx, f'0{n_qubits}b'), amp)

# Extraction de la solution (phase et ancilla à 0)
solution = []
for idx, amp in enumerate(state):
    binaire = format(idx, f'0{n_qubits}b')
    if binaire[:2] == '00':
        solution.append(amp)
solution = np.array(solution)
norm = np.linalg.norm(solution)
if norm > 0:
    solution = solution / norm
else:
    print("Aucune amplitude extraite, vérifiez la correspondance phase/valeur propre et la QPE.")

print("Vecteur solution normalisé (approximation HHL Qiskit, réseau électrique) :")
print(solution)

# Solution classique (matrice décalée)
sol_classique = np.linalg.solve(A_shifted, np.array([1, -1]))
print("Solution classique (matrice décalée) :", sol_classique)
sol_classique_norm = sol_classique / np.linalg.norm(sol_classique)
print("Solution classique normalisée :", sol_classique_norm)

Amplitudes non nulles :
000 (0.3411172891293404+0.15079483817311104j)
001 (0.14562078871610526-0.06584875225225037j)
010 (-0.10717980251862025-0.047380069821360414j)
011 (-0.31777061288129693+0.14369375791158626j)
100 (0.3487603282840282+0.4523845145193328j)
101 (0.1422832434934132-0.1975462567567506j)
110 (-0.10958126223158984-0.14214020946408115j)
111 (-0.3104874921106661+0.43108127373475763j)
Vecteur solution normalisé (approximation HHL Qiskit, réseau électrique) :
[0.84068635+0.37163511j 0.35888362-0.16228479j]
Solution classique (matrice décalée) : [ 0.19566693 -0.46817064]
Solution classique normalisée : [ 0.3856157  -0.92265949]
